
## Quick Run

This notebook is publicly available for any usage at our data imputation project. Please click [**transdim**](https://github.com/xinychen/transdim).


We start by importing the necessary dependencies. We will make use of `numpy`.

In [46]:
import numpy as np
from numpy.linalg import inv as inv

# Part 1: Matrix Computation Concepts

## 1) Kronecker product

- **Definition**:

Given two matrices $A\in\mathbb{R}^{m_1\times n_1}$ and $B\in\mathbb{R}^{m_2\times n_2}$, then, the **Kronecker product** between these two matrices is defined as

$$A\otimes B=\left[ \begin{array}{cccc} a_{11}B & a_{12}B & \cdots & a_{1m_2}B \\ a_{21}B & a_{22}B & \cdots & a_{2m_2}B \\ \vdots & \vdots & \ddots & \vdots \\ a_{m_11}B & a_{m_12}B & \cdots & a_{m_1m_2}B \\ \end{array} \right]$$
where the symbol $\otimes$ denotes Kronecker product, and the size of resulted $A\otimes B$ is $(m_1m_2)\times (n_1n_2)$ (i.e., $m_1\times m_2$ columns and $n_1\times n_2$ rows).

- **Example**:

If $A=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]$ and $B=\left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10 \\ \end{array} \right]$, then, we have

$$A\otimes B=\left[ \begin{array}{cc} 1\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] & 2\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] \\ 3\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] & 4\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] \\ \end{array} \right]$$

$$=\left[ \begin{array}{cccccc} 5 & 6 & 7 & 10 & 12 & 14 \\ 8 & 9 & 10 & 16 & 18 & 20 \\ 15 & 18 & 21 & 20 & 24 & 28 \\ 24 & 27 & 30 & 32 & 36 & 40 \\ \end{array} \right]\in\mathbb{R}^{4\times 6}.$$

## 2) Khatri-Rao product (`kr_prod`)

- **Definition**:

Given two matrices $A=\left( \boldsymbol{a}_1,\boldsymbol{a}_2,...,\boldsymbol{a}_r \right)\in\mathbb{R}^{m\times r}$ and $B=\left( \boldsymbol{b}_1,\boldsymbol{b}_2,...,\boldsymbol{b}_r \right)\in\mathbb{R}^{n\times r}$ with same number of columns, then, the **Khatri-Rao product** (or **column-wise Kronecker product**) between $A$ and $B$ is given as follows,

$$A\odot B=\left( \boldsymbol{a}_1\otimes \boldsymbol{b}_1,\boldsymbol{a}_2\otimes \boldsymbol{b}_2,...,\boldsymbol{a}_r\otimes \boldsymbol{b}_r \right)\in\mathbb{R}^{(mn)\times r}$$
where the symbol $\odot$ denotes Khatri-Rao product, and $\otimes$ denotes Kronecker product.

- **Example**:

If $A=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]=\left( \boldsymbol{a}_1,\boldsymbol{a}_2 \right) $ and $B=\left[ \begin{array}{cc} 5 & 6 \\ 7 & 8 \\ 9 & 10 \\ \end{array} \right]=\left( \boldsymbol{b}_1,\boldsymbol{b}_2 \right) $, then, we have

$$A\odot B=\left( \boldsymbol{a}_1\otimes \boldsymbol{b}_1,\boldsymbol{a}_2\otimes \boldsymbol{b}_2 \right) $$

$$=\left[ \begin{array}{cc} \left[ \begin{array}{c} 1 \\ 3 \\ \end{array} \right]\otimes \left[ \begin{array}{c} 5 \\ 7 \\ 9 \\ \end{array} \right] & \left[ \begin{array}{c} 2 \\ 4 \\ \end{array} \right]\otimes \left[ \begin{array}{c} 6 \\ 8 \\ 10 \\ \end{array} \right] \\ \end{array} \right]$$

$$=\left[ \begin{array}{cc} 5 & 12 \\ 7 & 16 \\ 9 & 20 \\ 15 & 24 \\ 21 & 32 \\ 27 & 40 \\ \end{array} \right]\in\mathbb{R}^{6\times 2}.$$

In [47]:
def kr_prod(a, b):
    return np.einsum('ir, jr -> ijr', a, b).reshape(a.shape[0] * b.shape[0], -1)

In [48]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8], [9, 10]])
print(kr_prod(A, B))

[[ 5 12]
 [ 7 16]
 [ 9 20]
 [15 24]
 [21 32]
 [27 40]]


# Part 2: Temporal Regularized Matrix Factorization (TRMF)

Implementing TRMF involves learning three main components:

- spatial factor matrix
- temporal factor matrix
- coefficient matrices of AR

In this implementation, we have included Gibbs sampling over these main components and other parameters/hyperparameters.

In [49]:
def TRMF(dense_mat, sparse_mat, init, time_lags, multi_steps, lambda_w, lambda_x, lambda_theta, eta, maxiter):
    """Temporal Regularized Matrix Factorization, TRMF."""
    W = init["W"]
    X = init["X"]
    theta = init["theta"]
    
    dim1, dim2 = sparse_mat.shape
    binary_mat = np.zeros((dim1,dim2))
    position = np.where((sparse_mat != 0))
    binary_mat[position] = 1
    pos = np.where((dense_mat != 0) & (sparse_mat == 0))
    d = len(time_lags)
    r = theta.shape[1]

    mape = np.zeros(maxiter)
    rmse = np.zeros(maxiter)
    for iters in range(maxiter):
        var1 = X.T
        var2 = kr_prod(var1,var1)
        var3 = np.matmul(var2,binary_mat.T)
        var4 = np.matmul(var1,sparse_mat.T)
        for i in range(dim1):
            W[i,:] = np.matmul(inv((var3[:,i].reshape([r,r]))+lambda_w * np.eye(r)), var4[:,i])

        var1 = W.T
        var2 = kr_prod(var1,var1)
        var3 = np.matmul(var2, binary_mat)
        var4 = np.matmul(var1, sparse_mat)
        for t in range(dim2):
            Mt = np.zeros((r,r))
            Nt = np.zeros(r)
            if t < max(time_lags):
                Pt = np.zeros((r,r))
                Qt = np.zeros(r)
            else:
                Pt = np.eye(r)
                Qt = np.einsum('ij, ij -> j', theta, X[t - time_lags, :])
            if t < dim2 - np.min(time_lags):
                if t >= np.max(time_lags) and t < dim2 - np.max(time_lags):
                    index = list(range(0, d))
                else:
                    index = list(np.where((t + time_lags >= np.max(time_lags)) & (t + time_lags < dim2)))[0]
                for k in index:
                    theta0 = theta.copy()
                    theta0[k, :] = 0
                    Mt = Mt + np.diag(theta[k, :]**2);
                    Nt = Nt + np.multiply(theta[k,:],(X[t+time_lags[k], :] 
                                                                 - np.einsum('ij, ij -> j', theta0,
                                                                             X[t + time_lags[k] - time_lags, :])))
                X[t,:] = np.matmul(inv(var3[:, t].reshape([r,r])
                                       + lambda_x * Pt + lambda_x * Mt + lambda_x * eta * np.eye(r)),
                                   (var4[:, t] + lambda_x * Qt + lambda_x * Nt))
            elif t >= dim2 - np.min(time_lags):
                X[t, :] = np.matmul(inv(var3[:, t].reshape([r, r]) + lambda_x * Pt 
                                        + lambda_x * eta * np.eye(r)), (var4[:, t] + Qt))
        for k in range(d):
            var1 = X[np.max(time_lags) - time_lags[k] : dim2 - time_lags[k], :]
            var2 = inv(np.diag(np.einsum('ij, ij -> j', var1, var1)) + (lambda_theta / lambda_x) * np.eye(r))
            var3 = np.zeros(r)
            for t in range(np.max(time_lags) - time_lags[k], dim2 - time_lags[k]):
                var3 = var3 + np.multiply(X[t, :],
                                          (X[t + time_lags[k], :] 
                                           - np.einsum('ij, ij -> j', theta, X[t + time_lags[k] - time_lags, :])
                                           +np.multiply(theta[k, :], X[t,:])))
            theta[k, :] = np.matmul(var2,var3)

        mat_hat = np.matmul(W, X.T)
        rmse = np.sqrt(np.sum((dense_mat[pos] - mat_hat[pos]) ** 2)/dense_mat[pos].shape[0])
        if (iters + 1) % 200 == 0:
            print('Iter: {}'.format(iters + 1))
            print('RMSE: {:.6}'.format(rmse))
            print()

    X_new = np.zeros((dim2 + multi_steps, rank))
    X_new[0 : dim2, :] = X.copy()
    for t0 in range(multi_steps):
        X_new[dim2 + t0, :] = np.einsum('ij, ij -> j', theta, X_new[dim2 + t0 - time_lags, :])
    if maxiter >= 100:
        final_mape = np.sum(np.abs(dense_mat[pos] - mat_hat[pos])/dense_mat[pos])/dense_mat[pos].shape[0]
        final_rmse = np.sqrt(np.sum((dense_mat[pos] - mat_hat[pos]) ** 2)/dense_mat[pos].shape[0])
        print('Imputation MAPE: {:.6}'.format(final_mape))
        print('Imputation RMSE: {:.6}'.format(final_rmse))
        print()
    return np.matmul(W, X_new[dim2 : dim2 + multi_steps, :].T), W, X_new, theta

## Multi-step prediction

In the multi-step prediction task, to enable training data for each rolling step informative, we do not apply an online implementation anymore.

Involving rolling prediction tasks, there are two crucial inputs:

- **`pred_time_steps`**: the number of steps we should forecast, e.g., if we want to forecast time series within 5 days (144 time slots/steps per day) in advance, then the `pred_time_steps` is $5\times 144=720$;
- **`multi_steps`**: the number of steps we should forecast at the current step, e.g., if we want to forecast time series within 2 hours (7 time slots/steps per hour) in advance, then the `multi_stpes` is $2\times 6=12$.

In [50]:
def multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                     rank, pred_time_steps, multi_steps, maxiter):
    T = dense_mat.shape[1]
    start_time = T - pred_time_steps
    dim1 = dense_mat.shape[0]
    d = time_lags.shape[0]
    mat_hat = np.zeros((dim1, pred_time_steps))
    
    for t in range(int(pred_time_steps/multi_steps)):
        if t == 0:
            init = {"W": 0.1 * np.random.rand(dim1, rank), "X": 0.1 * np.random.rand(start_time, rank),
                   "theta": 0.1 * np.random.rand(d, rank)}
            mat, W, X, theta = TRMF(dense_mat[:, 0 : start_time], 
                                    sparse_mat[:, 0 : start_time], 
                                    init, time_lags, multi_steps, 
                                    lambda_w, lambda_x, lambda_theta, eta, maxiter[0])
        else:
            init = {"W": W, "X": X, "theta": theta}
            mat, W, X, theta = TRMF(dense_mat[:, 0 : start_time + t * multi_steps], 
                                    sparse_mat[:, 0 : start_time + t * multi_steps], 
                                    init, time_lags, multi_steps, 
                                    lambda_w, lambda_x, lambda_theta, eta, maxiter[1])
        mat_hat[:, t * multi_steps : (t + 1) * multi_steps] = mat[:, mat.shape[1] - multi_steps : mat.shape[1]]

    small_dense_mat = dense_mat[:, start_time : dense_mat.shape[1]]
    pos = np.where(small_dense_mat != 0)
    final_mape = np.sum(np.abs(small_dense_mat[pos] - 
                               mat_hat[pos])/small_dense_mat[pos])/small_dense_mat[pos].shape[0]
    final_rmse = np.sqrt(np.sum((small_dense_mat[pos] - 
                                 mat_hat[pos]) ** 2)/small_dense_mat[pos].shape[0])
    print('Final MAPE: {:.6}'.format(final_mape))
    print('Final RMSE: {:.6}'.format(final_rmse))
    print()
    return mat_hat

# Part 3: Data Organization

## 1) Matrix Structure

We consider a dataset of $m$ discrete time series $\boldsymbol{y}_{i}\in\mathbb{R}^{f},i\in\left\{1,2,...,m\right\}$. The time series may have missing elements. We express spatio-temporal dataset as a matrix $Y\in\mathbb{R}^{m\times f}$ with $m$ rows (e.g., locations) and $f$ columns (e.g., discrete time intervals),

$$Y=\left[ \begin{array}{cccc} y_{11} & y_{12} & \cdots & y_{1f} \\ y_{21} & y_{22} & \cdots & y_{2f} \\ \vdots & \vdots & \ddots & \vdots \\ y_{m1} & y_{m2} & \cdots & y_{mf} \\ \end{array} \right]\in\mathbb{R}^{m\times f}.$$

## 2) Tensor Structure

We consider a dataset of $m$ discrete time series $\boldsymbol{y}_{i}\in\mathbb{R}^{nf},i\in\left\{1,2,...,m\right\}$. The time series may have missing elements. We partition each time series into intervals of predifined length $f$. We express each partitioned time series as a matrix $Y_{i}$ with $n$ rows (e.g., days) and $f$ columns (e.g., discrete time intervals per day),

$$Y_{i}=\left[ \begin{array}{cccc} y_{11} & y_{12} & \cdots & y_{1f} \\ y_{21} & y_{22} & \cdots & y_{2f} \\ \vdots & \vdots & \ddots & \vdots \\ y_{n1} & y_{n2} & \cdots & y_{nf} \\ \end{array} \right]\in\mathbb{R}^{n\times f},i=1,2,...,m,$$

therefore, the resulting structure is a tensor $\mathcal{Y}\in\mathbb{R}^{m\times n\times f}$.

# Part 4: Experiments on Guangzhou Data Set

In [51]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.0

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
                                                                   random_tensor.shape[1] 
                                                                   * random_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [52]:
import time
start = time.time()
pred_time_steps = 144 * 5
multi_steps = 144
time_lags = np.array([1, 2, 3, 144, 144+1, 144+2, 7*144, 7*144+1, 7*144+2])

rank = 10
lambda_w = 500
lambda_x = 500
lambda_theta = 500
eta = 0.03
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

/Users/xinyuchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in double_scalars
/Users/xinyuchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in double_scalars
/Users/xinyuchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in double_scalars


Imputation MAPE: nan
Imputation RMSE: nan

Final MAPE: 0.155713
Final RMSE: 5.86419

Running time: 353 seconds


In [53]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.2

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
                                                                   random_tensor.shape[1] 
                                                                   * random_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [54]:
import time
start = time.time()
pred_time_steps = 144 * 5
multi_steps = 144
time_lags = np.array([1, 2, 3, 144, 144+1, 144+2, 7*144, 7*144+1, 7*144+2])

rank = 10
lambda_w = 500
lambda_x = 500
lambda_theta = 500
eta = 0.03
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.0975444
Imputation RMSE: 4.10002

Final MAPE: 0.157781
Final RMSE: 5.90502

Running time: 520 seconds


In [55]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
                                                                   random_tensor.shape[1] 
                                                                   * random_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [56]:
import time
start = time.time()
pred_time_steps = 144 * 5
multi_steps = 144
time_lags = np.array([1, 2, 3, 144, 144+1, 144+2, 7*144, 7*144+1, 7*144+2])

rank = 10
lambda_w = 500
lambda_x = 500
lambda_theta = 500
eta = 0.03
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.098038
Imputation RMSE: 4.11825

Final MAPE: 0.1578
Final RMSE: 5.8566

Running time: 554 seconds


In [57]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.2

# =============================================================================
### Non-random missing (NM) scenario
### Set the NM scenario by:
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1,i2,:] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] 
                                    * binary_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [58]:
import time
start = time.time()
pred_time_steps = 144 * 5
multi_steps = 144
time_lags = np.array([1, 2, 3, 144, 144+1, 144+2, 7*144, 7*144+1, 7*144+2])

rank = 10
lambda_w = 500
lambda_x = 500
lambda_theta = 500
eta = 0.03
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.101409
Imputation RMSE: 4.26312

Final MAPE: 0.154032
Final RMSE: 5.72788

Running time: 546 seconds


In [59]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

# =============================================================================
### Non-random missing (NM) scenario
### Set the NM scenario by:
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1,i2,:] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] 
                                    * binary_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [60]:
import time
start = time.time()
pred_time_steps = 144 * 5
multi_steps = 144
time_lags = np.array([1, 2, 3, 144, 144+1, 144+2, 7*144, 7*144+1, 7*144+2])

rank = 10
lambda_w = 500
lambda_x = 500
lambda_theta = 500
eta = 0.03
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.102683
Imputation RMSE: 4.33797

Final MAPE: 0.160571
Final RMSE: 5.9881

Running time: 462 seconds


**Experiment results** of multi-step prediction with missing values using TRMF:

|  scenario |`rank`|`time_lags`| `maxiter` |       mape |      rmse |
|:----------|-----:|---------:|---------:|-----------:|----------:|
|**Original data**| 10 | (1,2,3,144,144+1,144+2,7$\times$144,7$\times$144+1,7$\times$144+2) | (100,10) | **0.1557** | **5.86**|
|**20%, RM**| 10 | (1,2,3,144,144+1,144+2,7$\times$144,7$\times$144+1,7$\times$144+2) | (100,10) | **0.1578** | **5.91**|
|**40%, RM**| 10 | (1,2,3,144,144+1,144+2,7$\times$144,7$\times$144+1,7$\times$144+2) | (100,10) | **0.1578** | **5.86**|
|**20%, NM**| 10 | (1,2,3,144,144+1,144+2,7$\times$144,7$\times$144+1,7$\times$144+2) | (100,10) | **0.1540** | **5.73**|
|**40%, NM**| 10 | (1,2,3,144,144+1,144+2,7$\times$144,7$\times$144+1,7$\times$144+2) | (100,10) | **0.1606** | **5.99**|


# Part 5: Experiments on Birmingham Data Set

In [61]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Birmingham-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.0

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
                                                                   random_tensor.shape[1] 
                                                                   * random_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [62]:
import time
start = time.time()
pred_time_steps = 18 * 7
multi_steps = 18
time_lags = np.array([1, 2, 3, 18, 18+1, 18+2, 7*18, 7*18+1, 7*18+2])

rank = 10
lambda_w = 100
lambda_x = 100
lambda_theta = 100
eta = 0.01
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

/Users/xinyuchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in double_scalars
/Users/xinyuchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in double_scalars
/Users/xinyuchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in double_scalars


Imputation MAPE: nan
Imputation RMSE: nan

Final MAPE: 0.277757
Final RMSE: 230.433

Running time: 58 seconds


In [63]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Birmingham-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.1

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
                                                                   random_tensor.shape[1] 
                                                                   * random_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [64]:
import time
start = time.time()
pred_time_steps = 18 * 7
multi_steps = 18
time_lags = np.array([1, 2, 3, 18, 18+1, 18+2, 7*18, 7*18+1, 7*18+2])

rank = 10
lambda_w = 100
lambda_x = 100
lambda_theta = 100
eta = 0.01
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.0826114
Imputation RMSE: 24.4381

Final MAPE: 0.275518
Final RMSE: 213.014

Running time: 58 seconds


In [65]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Birmingham-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.3

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
                                                                   random_tensor.shape[1] 
                                                                   * random_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [66]:
import time
start = time.time()
pred_time_steps = 18 * 7
multi_steps = 18
time_lags = np.array([1, 2, 3, 18, 18+1, 18+2, 7*18, 7*18+1, 7*18+2])

rank = 10
lambda_w = 100
lambda_x = 100
lambda_theta = 100
eta = 0.01
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.0826661
Imputation RMSE: 28.5482

Final MAPE: 0.340656
Final RMSE: 225.042

Running time: 58 seconds


In [67]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Birmingham-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.1

# =============================================================================
### Non-random missing (NM) scenario
### Set the NM scenario by:
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1,i2,:] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [68]:
import time
start = time.time()
pred_time_steps = 18 * 7
multi_steps = 18
time_lags = np.array([1, 2, 3, 18, 18+1, 18+2, 7*18, 7*18+1, 7*18+2])

rank = 10
lambda_w = 100
lambda_x = 100
lambda_theta = 100
eta = 0.01
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.127057
Imputation RMSE: 28.5368

Final MAPE: 0.26932
Final RMSE: 252.052

Running time: 58 seconds


In [69]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Birmingham-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.3

# =============================================================================
### Non-random missing (NM) scenario
### Set the NM scenario by:
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1,i2,:] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [70]:
import time
start = time.time()
pred_time_steps = 18 * 7
multi_steps = 18
time_lags = np.array([1, 2, 3, 18, 18+1, 18+2, 7*18, 7*18+1, 7*18+2])

rank = 10
lambda_w = 100
lambda_x = 100
lambda_theta = 100
eta = 0.01
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.198753
Imputation RMSE: 153.063

Final MAPE: 0.268799
Final RMSE: 192.529

Running time: 58 seconds


**Experiment results** of multi-step prediction with missing values using TRMF:

|  scenario |`rank`|`time_lags`| `maxiter` |       mape |      rmse |
|:----------|-----:|---------:|---------:|-----------:|----------:|
|**Original data**| 10 | (1,2,3,18,18+1,18+2,7$\times$18,7$\times$18+1,7$\times$18+2) | (100,10) | **0.2778** | **230.43**|
|**10%, RM**| 10 | (1,2,3,18,18+1,18+2,7$\times$18,7$\times$18+1,7$\times$18+2) | (100,10) | **0.2755** | **213.01**|
|**30%, RM**| 10 | (1,2,3,18,18+1,18+2,7$\times$18,7$\times$18+1,7$\times$18+2) | (100,10) | **0.3407** | **225.04**|
|**10%, NM**| 10 | (1,2,3,18,18+1,18+2,7$\times$18,7$\times$18+1,7$\times$18+2) | (100,10) | **0.2693** | **252.05**|
|**30%, NM**| 10 | (1,2,3,18,18+1,18+2,7$\times$18,7$\times$18+1,7$\times$18+2) | (100,10) | **0.2688** | **192.53**|


# Part 6: Experiments on Hangzhou Data Set

In [71]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.0

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
                                                                   random_tensor.shape[1] 
                                                                   * random_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [72]:
import time
start = time.time()
pred_time_steps = 108 * 5
multi_steps = 108
time_lags = np.array([1, 2, 3, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])

rank = 10
lambda_w = 1000
lambda_x = 1000
lambda_theta = 1000
eta = 0.05
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

/Users/xinyuchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in double_scalars
/Users/xinyuchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in double_scalars
/Users/xinyuchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in double_scalars


Imputation MAPE: nan
Imputation RMSE: nan

Final MAPE: 0.247562
Final RMSE: 39.9642

Running time: 72 seconds


In [73]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.2

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
                                                                   random_tensor.shape[1] 
                                                                   * random_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [74]:
import time
start = time.time()
pred_time_steps = 108 * 5
multi_steps = 108
time_lags = np.array([1, 2, 3, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])

rank = 10
lambda_w = 1000
lambda_x = 1000
lambda_theta = 1000
eta = 0.05
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.226878
Imputation RMSE: 35.9248

Final MAPE: 0.249323
Final RMSE: 38.3597

Running time: 72 seconds


In [75]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
                                                                   random_tensor.shape[1] 
                                                                   * random_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [76]:
import time
start = time.time()
pred_time_steps = 108 * 5
multi_steps = 108
time_lags = np.array([1, 2, 3, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])

rank = 10
lambda_w = 1000
lambda_x = 1000
lambda_theta = 1000
eta = 0.05
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.231478
Imputation RMSE: 39.4553

Final MAPE: 0.260893
Final RMSE: 39.6213

Running time: 72 seconds


In [77]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.2

# =============================================================================
### Non-random missing (NM) scenario
### Set the NM scenario by:
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1,i2,:] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] 
                                    * binary_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [78]:
import time
start = time.time()
pred_time_steps = 108 * 5
multi_steps = 108
time_lags = np.array([1, 2, 3, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])

rank = 10
lambda_w = 1000
lambda_x = 1000
lambda_theta = 1000
eta = 0.05
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.250741
Imputation RMSE: 44.4853

Final MAPE: 0.23286
Final RMSE: 39.885

Running time: 73 seconds


In [79]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

# =============================================================================
### Non-random missing (NM) scenario
### Set the NM scenario by:
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1,i2,:] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] 
                                    * binary_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [80]:
import time
start = time.time()
pred_time_steps = 108 * 5
multi_steps = 108
time_lags = np.array([1, 2, 3, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])

rank = 10
lambda_w = 1000
lambda_x = 1000
lambda_theta = 1000
eta = 0.05
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.25825
Imputation RMSE: 43.1692

Final MAPE: 0.258291
Final RMSE: 44.4489

Running time: 73 seconds


**Experiment results** of multi-step prediction with missing values using TRMF:

|  scenario |`rank`|`time_lags`| `maxiter` |       mape |      rmse |
|:----------|-----:|---------:|---------:|-----------:|----------:|
|**Original data**| 10 | (1,2,3,108,108+1,108+2,7$\times$108,7$\times$108+1,7$\times$108+2) | (100,10) | **0.2476** | **39.96**|
|**20%, RM**| 10 | (1,2,3,108,108+1,108+2,7$\times$108,7$\times$108+1,7$\times$108+2) | (100,10) | **0.2493** | **38.36**|
|**40%, RM**| 10 | (1,2,3,108,108+1,108+2,7$\times$108,7$\times$108+1,7$\times$108+2) | (100,10) | **0.2609** | **39.62**|
|**20%, NM**| 10 | (1,2,3,108,108+1,108+2,7$\times$108,7$\times$108+1,7$\times$108+2) | (100,10) | **0.2329** | **39.89**|
|**40%, NM**| 10 | (1,2,3,108,108+1,108+2,7$\times$108,7$\times$108+1,7$\times$108+2) | (100,10) | **0.2583** | **44.45**|


# Part 7: Experiments on Seattle Data Set

In [81]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
RM_mat = pd.read_csv('../datasets/Seattle-data-set/RM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
RM_mat = RM_mat.values

missing_rate = 0.0
# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(RM_mat + 0.5 - missing_rate)
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [82]:
import time
start = time.time()
pred_time_steps = 288 * 5
multi_steps = 288
time_lags = np.array([1, 2, 3, 288, 288+1, 288+2, 7*288, 7*288+1, 7*288+2])

rank = 10
lambda_w = 500
lambda_x = 500
lambda_theta = 500
eta = 0.03
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.0865804
Imputation RMSE: 4.92553

Final MAPE: 0.190956
Final RMSE: 9.93094

Running time: 234 seconds


In [83]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
RM_mat = pd.read_csv('../datasets/Seattle-data-set/RM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
RM_mat = RM_mat.values

missing_rate = 0.2
# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(RM_mat + 0.5 - missing_rate)
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [84]:
import time
start = time.time()
pred_time_steps = 288 * 5
multi_steps = 288
time_lags = np.array([1, 2, 3, 288, 288+1, 288+2, 7*288, 7*288+1, 7*288+2])

rank = 10
lambda_w = 500
lambda_x = 500
lambda_theta = 500
eta = 0.03
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.0865707
Imputation RMSE: 4.98909

Final MAPE: 0.189899
Final RMSE: 10.1037

Running time: 238 seconds


In [85]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
RM_mat = pd.read_csv('../datasets/Seattle-data-set/RM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
RM_mat = RM_mat.values

missing_rate = 0.4
# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(RM_mat + 0.5 - missing_rate)
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [86]:
import time
start = time.time()
pred_time_steps = 288 * 5
multi_steps = 288
time_lags = np.array([1, 2, 3, 288, 288+1, 288+2, 7*288, 7*288+1, 7*288+2])

rank = 10
lambda_w = 500
lambda_x = 500
lambda_theta = 500
eta = 0.03
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.0876804
Imputation RMSE: 5.03102

Final MAPE: 0.178549
Final RMSE: 9.44839

Running time: 239 seconds


In [87]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
NM_mat = pd.read_csv('../datasets/Seattle-data-set/NM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
NM_mat = NM_mat.values

missing_rate = 0.2
# =============================================================================
### Non-random missing (NM) scenario
### Set the NM scenario by:
binary_tensor = np.zeros((dense_mat.shape[0], 28, 288))
for i1 in range(binary_tensor.shape[0]):
    for i2 in range(binary_tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(NM_mat[i1, i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [88]:
import time
start = time.time()
pred_time_steps = 288 * 5
multi_steps = 288
time_lags = np.array([1, 2, 3, 288, 288+1, 288+2, 7*288, 7*288+1, 7*288+2])

rank = 10
lambda_w = 500
lambda_x = 500
lambda_theta = 500
eta = 0.03
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.094001
Imputation RMSE: 5.32562

Final MAPE: 0.184133
Final RMSE: 9.70225

Running time: 238 seconds


In [89]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
NM_mat = pd.read_csv('../datasets/Seattle-data-set/NM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
NM_mat = NM_mat.values

missing_rate = 0.4
# =============================================================================
### Non-random missing (NM) scenario
### Set the NM scenario by:
binary_tensor = np.zeros((dense_mat.shape[0], 28, 288))
for i1 in range(binary_tensor.shape[0]):
    for i2 in range(binary_tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(NM_mat[i1, i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [90]:
import time
start = time.time()
pred_time_steps = 288 * 5
multi_steps = 288
time_lags = np.array([1, 2, 3, 288, 288+1, 288+2, 7*288, 7*288+1, 7*288+2])

rank = 10
lambda_w = 500
lambda_x = 500
lambda_theta = 500
eta = 0.03
d = time_lags.shape[0]

maxiter = np.array([100, 10])
small_dense_mat = dense_mat[:, dense_mat.shape[1] - pred_time_steps : dense_mat.shape[1]]
mat_hat = multi_prediction(dense_mat, sparse_mat, time_lags, lambda_w, lambda_x, lambda_theta, eta, 
                           rank, pred_time_steps, multi_steps, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Imputation MAPE: 0.0939846
Imputation RMSE: 5.3489

Final MAPE: 0.170951
Final RMSE: 9.07586

Running time: 239 seconds


**Experiment results** of multi-step prediction with missing values using TRMF:

|  scenario |`rank`|`time_lags`| `maxiter` |       mape |      rmse |
|:----------|-----:|---------:|---------:|-----------:|----------:|
|**Original data**| 10 | (1,2,3,288,288+1,288+2,7$\times$288,7$\times$288+1,7$\times$288+2) | (100,10) | **0.1910** | **9.93**|
|**20%, RM**| 10 | (1,2,3,288,288+1,288+2,7$\times$288,7$\times$288+1,7$\times$288+2) | (100,10) | **0.1899** | **10.10**|
|**40%, RM**| 10 | (1,2,3,288,288+1,288+2,7$\times$288,7$\times$288+1,7$\times$288+2) | (100,10) | **0.1785** | **9.45**|
|**20%, NM**| 10 | (1,2,3,288,288+1,288+2,7$\times$288,7$\times$288+1,7$\times$288+2) | (100,10) | **0.1841** | **9.70**|
|**40%, NM**| 10 | (1,2,3,288,288+1,288+2,7$\times$288,7$\times$288+1,7$\times$288+2) | (100,10) | **0.1710** | **9.08**|
